# Base Metals Market Balance Commentary Using WBMS RICs

This notebook reads the latest monthly production and consumption data for following six base metals from WBMS RICS and produces a text file with commentary on the latest market balance.
1. Aluminium
2. Copper
3. Lead
4. Nickel
5. Tin
6. Zinc

### Import Required Libraries

In [1]:
import pandas as pd
import refinitiv.data as rd
import calendar
rd.open_session()

<refinitiv.data.session.Definition object at 0x162a9e790 {name='workspace'}>

## Define Helper Functions

In the code below, we define four helper functions. Three of them are templates for the commentary and the last function converts numbers to words:

1. sent_non_jan_non_dec():
   Defines template for all months except January and December.

2. sent_jan():
   Defines template for month of January.
3. sent_dec():
   Defines template for month of December.
4. num2words():
   Function to convert number to word.

In [2]:
def sent_non_jan_non_dec(metal_name, month_from, month_to, production, consumption, deficit_surplus, market_balance, metal, howmanymonths, prev_mkt_bal, prev_def_sur, prev_year_whole, prev_year_def_sur):
    text1 = metal_name+' output for the '+month_from+' to '+month_to+' period was '+production+' tonnes compared to apparent consumption of '+consumption + \
        ' tonnes, producing an implied market '+deficit_surplus+' of '+market_balance + \
        ' tonnes in the '+' market for the first ' + \
        howmanymonths+' months of the current year*.'
    text2 = 'The '+deficit_surplus+' of '+market_balance+" tonnes "+'compares with a '+prev_def_sur+' of '+prev_mkt_bal+" tonnes " + \
        " for the same period last year and a "+prev_year_def_sur+" of " + \
            prev_year_whole+" tonnes "+"for the previous year as a whole*."
    return text1+text2


def sent_jan(metal_name, production, consumption, deficit_surplus, market_balance, prev_mkt_bal, prev_def_sur, prev_year_whole, prev_year_def_sur):
    text1 = metal_name+' output for the January period, was '+production+' tonnes compared to apparent consumption of '+consumption + \
        ' tonnes, producing an implied market '+deficit_surplus+' of '+market_balance + \
        ' tonnes in the market for the first month of the current year.'
    text2 = '\n'+'The '+market_balance+" tonnes "+deficit_surplus+' compares with '+prev_mkt_bal+" tonnes "+prev_def_sur + \
        " for the same period last year and a "+prev_year_whole + \
            " "+prev_year_def_sur+" for the previous year as a whole."
    return text1  # +text2


def sent_dec(metal_name, month_to, production, consumption, deficit_surplus, market_balance, prev_year_whole, prev_year_def_sur):
    text1 = metal_name+' output for the January'+' to '+month_to+' period, was '+production+' tonnes compared to apparent consumption of ' + \
        consumption+' tonnes, producing an implied market '+deficit_surplus + \
        ' of '+market_balance+' tonnes in the market for the current year.'
    text2 = '\n'+'The '+market_balance+" tonnes "+deficit_surplus+' compares with ' + \
        prev_year_whole+" tonnes "+prev_year_def_sur+" for the previous year."
    return text1+text2


def num2words(numb):
    num2word = {1: 'one', 2: 'two', 3: 'three', 4: 'four', 5: 'five',
                6: 'six', 7: 'seven', 8: 'eight', 9: 'nine', 10: 'ten',
                11: 'eleven', 12: 'twelve'}
    return num2word[numb]

## Aluminium Data Preperation

The below code fetches production and consumption numbers from WBMS RICs and performs the requisite calculations needed to generate market balance commentary for Aluminium.

In [3]:
import gc
df_prod = rd.get_history(
    "WBMALPRSPALWWM", start='2015-01-01', interval='monthly', count=10000)
df_prod = df_prod.reset_index()
df_cons = rd.get_history(
    "WBMALCONPALWWM", start='2015-01-01', interval='monthly', count=10000)
df_cons = df_cons.reset_index()
df_market_bal = pd.merge(df_prod, df_cons, on='Date')
del df_prod, df_cons
gc.collect()
df_market_bal = df_market_bal.sort_values('Date')
df_market_bal['Year'] = df_market_bal['Date'].dt.year
df_market_bal['Month'] = df_market_bal['Date'].apply(
    lambda x: str(x).split('-')[1:2][0].replace(' 00:00:00', ''))
df_market_bal['IntMonth'] = df_market_bal['Month'].apply(lambda x: int(x))
df_market_bal = df_market_bal.sort_values(by=['Year', 'IntMonth'])
metal_name = 'Primary Aluminium'
metal = 'Aluminium'

current_month_num = df_market_bal.iloc[-1]['Month']
current_year = df_market_bal.iloc[-1]['Year']
last_year = current_year-1

if current_month_num not in ['11', '12']:
    howmanymonths = num2words(len(range(int(current_month_num))))
else:
    howmanymonths = current_month_num

df_market_bal['MarketBalance'] = df_market_bal['TRDPRC_1_x'] - \
    df_market_bal['TRDPRC_1_y']
df_market_bal['YTDProd'] = df_market_bal.groupby(['Year'])[
    'TRDPRC_1_x'].cumsum()
df_market_bal['YTDCons'] = df_market_bal.groupby(['Year'])[
    'TRDPRC_1_y'].cumsum()
df_market_bal['YTDMarketBal'] = df_market_bal['YTDProd'] - \
    df_market_bal['YTDCons']
production = "{:,}".format((df_market_bal.iloc[-1]['YTDProd']))
consumption = "{:,}".format((df_market_bal.iloc[-1]['YTDCons']))
market_balance = df_market_bal.iloc[-1]['YTDMarketBal']
deficit_surplus = 'surplus'
if market_balance < 0:
    deficit_surplus = 'deficit'
market_balance = "{:,}".format(abs(market_balance))
prev_mkt_bal = df_market_bal[(df_market_bal.Month == current_month_num) & (
    df_market_bal['Year'] == last_year)]['YTDMarketBal'].values[0]
prev_def_sur = 'surplus'
if prev_mkt_bal < 0:
    prev_def_sur = 'deficit'
prev_mkt_bal = "{:,}".format((abs(prev_mkt_bal)))
prev_year_whole = df_market_bal[(
    df_market_bal['Year'] == last_year)]['MarketBalance'].sum()
prev_year_def_sur = 'surplus'
if prev_year_whole < 0:
    prev_year_def_sur = 'deficit'
prev_year_whole = "{:,}".format(abs(prev_year_whole))

The below code calls the relevant template for Aluminium market balance commentary based on the month.

In [4]:
if current_month_num == '01':
    fin_text = sent_jan(metal_name, production, consumption, deficit_surplus,
                        market_balance, prev_mkt_bal, prev_def_sur, prev_year_whole, prev_year_def_sur)
elif current_month_num == '12':
    fin_text = sent_dec(metal_name, calendar.month_name[int(
        current_month_num)], production, consumption, deficit_surplus, market_balance, prev_year_whole, prev_year_def_sur)
else:
    fin_text = (sent_non_jan_non_dec(metal_name, 'January', calendar.month_name[int(
        current_month_num)], production, consumption, deficit_surplus, market_balance, metal, howmanymonths, prev_mkt_bal, prev_def_sur, prev_year_whole, prev_year_def_sur))
fin_text_aluminium = fin_text.replace(".0", "")
fin_text_aluminium

'Primary Aluminium output for the January to December period, was 68,304,556 tonnes compared to apparent consumption of 68,448,583 tonnes, producing an implied market deficit of 144,027 tonnes in the market for the current year.\nThe 144,027 tonnes deficit compares with 1,161,450 tonnes deficit for the previous year.'

## Copper Data Preperation

The below code fetches production and consumption numbers from WBMS RICs and performs the requisite calculations needed to generate market balance commentary for Copper.

In [5]:
import gc
df_prod = rd.get_history(
    "WBMCUPRDREFWWM", start='2015-01-01', interval='monthly', count=10000)
df_prod = df_prod.reset_index()
df_cons = rd.get_history(
    "WBMCUCONREFWWM", start='2015-01-01', interval='monthly', count=10000)
df_cons = df_cons.reset_index()
df_market_bal = pd.merge(df_prod, df_cons, on='Date')
del df_prod, df_cons
gc.collect()
df_market_bal = df_market_bal.sort_values('Date')
df_market_bal['Year'] = df_market_bal['Date'].dt.year
df_market_bal['Month'] = df_market_bal['Date'].apply(
    lambda x: str(x).split('-')[1:2][0].replace(' 00:00:00', ''))
df_market_bal['IntMonth'] = df_market_bal['Month'].apply(lambda x: int(x))
df_market_bal = df_market_bal.sort_values(by=['Year', 'IntMonth'])
metal_name = 'Refined Copper'
metal = 'Copper'

current_month_num = df_market_bal.iloc[-1]['Month']
current_year = df_market_bal.iloc[-1]['Year']
last_year = current_year-1

if current_month_num not in ['11', '12']:
    howmanymonths = num2words(len(range(int(current_month_num))))
else:
    howmanymonths = current_month_num

df_market_bal['MarketBalance'] = df_market_bal['TRDPRC_1_x'] - \
    df_market_bal['TRDPRC_1_y']
df_market_bal['YTDProd'] = df_market_bal.groupby(['Year'])[
    'TRDPRC_1_x'].cumsum()
df_market_bal['YTDCons'] = df_market_bal.groupby(['Year'])[
    'TRDPRC_1_y'].cumsum()
df_market_bal['YTDMarketBal'] = df_market_bal['YTDProd'] - \
    df_market_bal['YTDCons']
production = "{:,}".format((df_market_bal.iloc[-1]['YTDProd']))
consumption = "{:,}".format((df_market_bal.iloc[-1]['YTDCons']))
market_balance = df_market_bal.iloc[-1]['YTDMarketBal']
deficit_surplus = 'surplus'
if market_balance < 0:
    deficit_surplus = 'deficit'
market_balance = "{:,}".format(abs(market_balance))
prev_mkt_bal = df_market_bal[(df_market_bal.Month == current_month_num) & (
    df_market_bal['Year'] == last_year)]['YTDMarketBal'].values[0]
prev_def_sur = 'surplus'
if prev_mkt_bal < 0:
    prev_def_sur = 'deficit'
prev_mkt_bal = "{:,}".format((abs(prev_mkt_bal)))
prev_year_whole = df_market_bal[(
    df_market_bal['Year'] == last_year)]['MarketBalance'].sum()
prev_year_def_sur = 'surplus'
if prev_year_whole < 0:
    prev_year_def_sur = 'deficit'
prev_year_whole = "{:,}".format(abs(prev_year_whole))

The below code calls the relevant template for copper market balance commentary based on the month.

In [6]:
if current_month_num == '01':
    fin_text = sent_jan(metal_name, 'January', calendar.month_name[int(
        current_month_num)], production, consumption, deficit_surplus, market_balance, metal, howmanymonths, prev_mkt_bal, prev_def_sur, prev_year_whole)
elif current_month_num == '12':
    fin_text = sent_dec(metal_name, calendar.month_name[int(
        current_month_num)], production, consumption, deficit_surplus, market_balance, prev_year_whole, prev_year_def_sur)
else:
    fin_text = (sent_non_jan_non_dec(metal_name, 'January', calendar.month_name[int(
        current_month_num)], production, consumption, deficit_surplus, market_balance, metal, howmanymonths, prev_mkt_bal, prev_def_sur, prev_year_whole, prev_year_def_sur))
fin_text_copper = fin_text.replace(".0", "")
fin_text_copper

'Refined Copper output for the January to December period, was 25,084,821 tonnes compared to apparent consumption of 25,991,799 tonnes, producing an implied market deficit of 906,978 tonnes in the market for the current year.\nThe 906,978 tonnes deficit compares with 472,580 tonnes deficit for the previous year.'

## Lead Data Preperation

The below code fetches production and consumption numbers from WBMS RICs and performs the requisite calculations needed to generate market balance commentary for Lead.

In [7]:
import gc
df_prod = rd.get_history(
    "WBMPBPRDRELWWM", start='2015-01-01', interval='monthly', count=10000)
df_prod = df_prod.reset_index()
df_cons = rd.get_history(
    "WBMPBCONRELWWM", start='2015-01-01', interval='monthly', count=10000)
df_cons = df_cons.reset_index()
df_market_bal = pd.merge(df_prod, df_cons, on='Date')
del df_prod, df_cons
gc.collect()
df_market_bal = df_market_bal.sort_values('Date')


df_market_bal['Year'] = df_market_bal['Date'].dt.year
df_market_bal['Month'] = df_market_bal['Date'].apply(
    lambda x: str(x).split('-')[1:2][0].replace(' 00:00:00', ''))
df_market_bal['IntMonth'] = df_market_bal['Month'].apply(lambda x: int(x))
df_market_bal = df_market_bal.sort_values(by=['Year', 'IntMonth'])
metal_name = 'Refined Lead'
metal = 'Lead'

current_month_num = df_market_bal.iloc[-1]['Month']
current_year = df_market_bal.iloc[-1]['Year']
last_year = current_year-1

if current_month_num not in ['11', '12']:
    howmanymonths = num2words(len(range(int(current_month_num))))
else:
    howmanymonths = current_month_num

df_market_bal['MarketBalance'] = df_market_bal['TRDPRC_1_x'] - \
    df_market_bal['TRDPRC_1_y']
df_market_bal['YTDProd'] = df_market_bal.groupby(['Year'])[
    'TRDPRC_1_x'].cumsum()
df_market_bal['YTDCons'] = df_market_bal.groupby(['Year'])[
    'TRDPRC_1_y'].cumsum()
df_market_bal['YTDMarketBal'] = df_market_bal['YTDProd'] - \
    df_market_bal['YTDCons']
production = "{:,}".format((df_market_bal.iloc[-1]['YTDProd']))
consumption = "{:,}".format((df_market_bal.iloc[-1]['YTDCons']))
market_balance = df_market_bal.iloc[-1]['YTDMarketBal']
deficit_surplus = 'surplus'
if market_balance < 0:
    deficit_surplus = 'deficit'
market_balance = "{:,}".format(abs(market_balance))
prev_mkt_bal = df_market_bal[(df_market_bal.Month == current_month_num) & (
    df_market_bal['Year'] == last_year)]['YTDMarketBal'].values[0]
prev_def_sur = 'surplus'
if prev_mkt_bal < 0:
    prev_def_sur = 'deficit'
prev_mkt_bal = "{:,}".format((abs(prev_mkt_bal)))
prev_year_whole = df_market_bal[(
    df_market_bal['Year'] == last_year)]['MarketBalance'].sum()
prev_year_def_sur = 'surplus'
if prev_year_whole < 0:
    prev_year_def_sur = 'deficit'
prev_year_whole = "{:,}".format(abs(prev_year_whole))

The below code calls the relevant template for Lead market balance commentary based on the month.

In [8]:
if current_month_num == '01':
    fin_text = sent_jan(metal_name, 'January', calendar.month_name[int(
        current_month_num)], production, consumption, deficit_surplus, market_balance, metal, howmanymonths, prev_mkt_bal, prev_def_sur, prev_year_whole)
elif current_month_num == '12':
    fin_text = sent_dec(metal_name, calendar.month_name[int(
        current_month_num)], production, consumption, deficit_surplus, market_balance, prev_year_whole, prev_year_def_sur)
else:
    fin_text = (sent_non_jan_non_dec(metal_name, 'January', calendar.month_name[int(
        current_month_num)], production, consumption, deficit_surplus, market_balance, metal, howmanymonths, prev_mkt_bal, prev_def_sur, prev_year_whole, prev_year_def_sur))
fin_text_lead = fin_text.replace(".0", "")
fin_text_lead

'Refined Lead output for the January to December period, was 14,822,948 tonnes compared to apparent consumption of 14,898,715 tonnes, producing an implied market deficit of 75,767 tonnes in the market for the current year.\nThe 75,767 tonnes deficit compares with 131,854 tonnes deficit for the previous year.'

## Nickel Data Preperation

The below code fetches production and consumption numbers from WBMS RICs and performs the requisite calculations needed to generate market balance commentary for Nickel.

In [9]:
import gc
df_prod = rd.get_history(
    "WBMNIPRRPRNWWM", start='2015-01-01', interval='monthly', count=10000)
df_prod = df_prod.reset_index()
df_cons = rd.get_history(
    "WBMNICONPNWWM", start='2015-01-01', interval='monthly', count=10000)
df_cons = df_cons.reset_index()
df_market_bal = pd.merge(df_prod, df_cons, on='Date')
del df_prod, df_cons
gc.collect()

df_market_bal = df_market_bal.sort_values('Date')


df_market_bal['Year'] = df_market_bal['Date'].dt.year
df_market_bal['Month'] = df_market_bal['Date'].apply(
    lambda x: str(x).split('-')[1:2][0].replace(' 00:00:00', ''))
df_market_bal['IntMonth'] = df_market_bal['Month'].apply(lambda x: int(x))
df_market_bal = df_market_bal.sort_values(by=['Year', 'IntMonth'])
metal_name = 'Primary Refined Nickel'
metal = 'Nickel'

current_month_num = df_market_bal.iloc[-1]['Month']
current_year = df_market_bal.iloc[-1]['Year']
last_year = current_year-1

if current_month_num not in ['11', '12']:
    howmanymonths = num2words(len(range(int(current_month_num))))
else:
    howmanymonths = current_month_num

df_market_bal['MarketBalance'] = df_market_bal['TRDPRC_1_x'] - \
    df_market_bal['TRDPRC_1_y']
df_market_bal['YTDProd'] = df_market_bal.groupby(['Year'])[
    'TRDPRC_1_x'].cumsum()
df_market_bal['YTDCons'] = df_market_bal.groupby(['Year'])[
    'TRDPRC_1_y'].cumsum()
df_market_bal['YTDMarketBal'] = df_market_bal['YTDProd'] - \
    df_market_bal['YTDCons']
production = "{:,}".format((df_market_bal.iloc[-1]['YTDProd']))
consumption = "{:,}".format((df_market_bal.iloc[-1]['YTDCons']))
market_balance = df_market_bal.iloc[-1]['YTDMarketBal']
deficit_surplus = 'surplus'
if market_balance < 0:
    deficit_surplus = 'deficit'
market_balance = "{:,}".format(abs(market_balance))
prev_mkt_bal = df_market_bal[(df_market_bal.Month == current_month_num) & (
    df_market_bal['Year'] == last_year)]['YTDMarketBal'].values[0]
prev_def_sur = 'surplus'
if prev_mkt_bal < 0:
    prev_def_sur = 'deficit'
prev_mkt_bal = "{:,}".format((abs(prev_mkt_bal)))
prev_year_whole = df_market_bal[(
    df_market_bal['Year'] == last_year)]['MarketBalance'].sum()
prev_year_def_sur = 'surplus'
if prev_year_whole < 0:
    prev_year_def_sur = 'deficit'
prev_year_whole = "{:,}".format(abs(prev_year_whole))

The below code calls the relevant template for Nickel market balance commentary based on the month.

In [10]:
if current_month_num == '01':
    fin_text = sent_jan(metal_name, 'January', calendar.month_name[int(
        current_month_num)], production, consumption, deficit_surplus, market_balance, metal, howmanymonths, prev_mkt_bal, prev_def_sur, prev_year_whole)
elif current_month_num == '12':
    fin_text = sent_dec(metal_name, calendar.month_name[int(
        current_month_num)], production, consumption, deficit_surplus, market_balance, prev_year_whole, prev_year_def_sur)
else:
    fin_text = (sent_non_jan_non_dec(metal_name, 'January', calendar.month_name[int(
        current_month_num)], production, consumption, deficit_surplus, market_balance, metal, howmanymonths, prev_mkt_bal, prev_def_sur, prev_year_whole, prev_year_def_sur))
fin_text_nickel = fin_text.replace(".0", "")
fin_text_nickel

'Primary Refined Nickel output for the January to December period, was 2,999,675 tonnes compared to apparent consumption of 3,060,243 tonnes, producing an implied market deficit of 60,568 tonnes in the market for the current year.\nThe 60,568 tonnes deficit compares with 146,828 tonnes deficit for the previous year.'

## Tin Data Preperation

The below code fetches production and consumption numbers from WBMS RICs and performs the requisite calculations needed to generate market balance commentary for Tin.

In [11]:
import gc
df_prod = rd.get_history(
    "WBMSNPRDRETWWM", start='2015-01-01', interval='monthly', count=10000)
df_prod = df_prod.reset_index()
df_cons = rd.get_history(
    "WBMSNCONRETWWM", start='2015-01-01', interval='monthly', count=10000)
df_cons = df_cons.reset_index()
df_market_bal = pd.merge(df_prod, df_cons, on='Date')
del df_prod, df_cons
gc.collect()
df_market_bal = df_market_bal.sort_values('Date')

df_market_bal['Year'] = df_market_bal['Date'].dt.year
df_market_bal['Month'] = df_market_bal['Date'].apply(
    lambda x: str(x).split('-')[1:2][0].replace(' 00:00:00', ''))
df_market_bal['IntMonth'] = df_market_bal['Month'].apply(lambda x: int(x))
df_market_bal = df_market_bal.sort_values(by=['Year', 'IntMonth'])
metal_name = 'Refined Tin'
metal = 'Tin'

current_month_num = df_market_bal.iloc[-1]['Month']
current_year = df_market_bal.iloc[-1]['Year']
last_year = current_year-1

if current_month_num not in ['11', '12']:
    howmanymonths = num2words(len(range(int(current_month_num))))
else:
    howmanymonths = current_month_num

df_market_bal['MarketBalance'] = df_market_bal['TRDPRC_1_x'] - \
    df_market_bal['TRDPRC_1_y']
df_market_bal['YTDProd'] = df_market_bal.groupby(['Year'])[
    'TRDPRC_1_x'].cumsum()
df_market_bal['YTDCons'] = df_market_bal.groupby(['Year'])[
    'TRDPRC_1_y'].cumsum()
df_market_bal['YTDMarketBal'] = df_market_bal['YTDProd'] - \
    df_market_bal['YTDCons']
production = "{:,}".format((df_market_bal.iloc[-1]['YTDProd']))
consumption = "{:,}".format((df_market_bal.iloc[-1]['YTDCons']))
market_balance = df_market_bal.iloc[-1]['YTDMarketBal']
deficit_surplus = 'surplus'
if market_balance < 0:
    deficit_surplus = 'deficit'
market_balance = "{:,}".format(abs(market_balance))
prev_mkt_bal = df_market_bal[(df_market_bal.Month == current_month_num) & (
    df_market_bal['Year'] == last_year)]['YTDMarketBal'].values[0]
prev_def_sur = 'surplus'
if prev_mkt_bal < 0:
    prev_def_sur = 'deficit'
prev_mkt_bal = "{:,}".format((abs(prev_mkt_bal)))
prev_year_whole = df_market_bal[(
    df_market_bal['Year'] == last_year)]['MarketBalance'].sum()
prev_year_def_sur = 'surplus'
if prev_year_whole < 0:
    prev_year_def_sur = 'deficit'
prev_year_whole = "{:,}".format(abs(prev_year_whole))

The below code calls the relevant template for Tin market balance commentary based on the month.

In [12]:
if current_month_num == '01':
    fin_text = sent_jan(metal_name, 'January', calendar.month_name[int(
        current_month_num)], production, consumption, deficit_surplus, market_balance, metal, howmanymonths, prev_mkt_bal, prev_def_sur, prev_year_whole)
elif current_month_num == '12':
    fin_text = sent_dec(metal_name, calendar.month_name[int(
        current_month_num)], production, consumption, deficit_surplus, market_balance, prev_year_whole, prev_year_def_sur)
else:
    fin_text = (sent_non_jan_non_dec(metal_name, 'January', calendar.month_name[int(
        current_month_num)], production, consumption, deficit_surplus, market_balance, metal, howmanymonths, prev_mkt_bal, prev_def_sur, prev_year_whole, prev_year_def_sur))
fin_text_tin = fin_text.replace(".0", "")
fin_text_tin

'Refined Tin output for the January to December period, was 348,932 tonnes compared to apparent consumption of 359,983 tonnes, producing an implied market deficit of 11,051 tonnes in the market for the current year.\nThe 11,051 tonnes deficit compares with 1,850 tonnes surplus for the previous year.'

## Zinc Data Preperation

The below code fetches production and consumption numbers from WBMS RICs and performs the requisite calculations needed to generate market balance commentary for Zinc.

In [13]:
import gc
df_prod = rd.get_history(
    "WBMZNPRDREZWWM", start='2015-01-01', interval='monthly', count=10000)
df_prod = df_prod.reset_index()
df_cons = rd.get_history(
    "WBMZNCONREZWWM", start='2015-01-01', interval='monthly', count=10000)
df_cons = df_cons.reset_index()
df_market_bal = pd.merge(df_prod, df_cons, on='Date')
del df_prod, df_cons
gc.collect()
df_market_bal = df_market_bal.sort_values('Date')

df_market_bal['Year'] = df_market_bal['Date'].dt.year
df_market_bal['Month'] = df_market_bal['Date'].apply(
    lambda x: str(x).split('-')[1:2][0].replace(' 00:00:00', ''))
df_market_bal['IntMonth'] = df_market_bal['Month'].apply(lambda x: int(x))
df_market_bal = df_market_bal.sort_values(by=['Year', 'IntMonth'])
metal_name = 'Refined Lead'
metal = 'Lead'

current_month_num = df_market_bal.iloc[-1]['Month']
current_year = df_market_bal.iloc[-1]['Year']
last_year = current_year-1

if current_month_num not in ['11', '12']:
    howmanymonths = num2words(len(range(int(current_month_num))))
else:
    howmanymonths = current_month_num

df_market_bal['MarketBalance'] = df_market_bal['TRDPRC_1_x'] - \
    df_market_bal['TRDPRC_1_y']
df_market_bal['YTDProd'] = df_market_bal.groupby(['Year'])[
    'TRDPRC_1_x'].cumsum()
df_market_bal['YTDCons'] = df_market_bal.groupby(['Year'])[
    'TRDPRC_1_y'].cumsum()
df_market_bal['YTDMarketBal'] = df_market_bal['YTDProd'] - \
    df_market_bal['YTDCons']
production = "{:,}".format((df_market_bal.iloc[-1]['YTDProd']))
consumption = "{:,}".format((df_market_bal.iloc[-1]['YTDCons']))
market_balance = df_market_bal.iloc[-1]['YTDMarketBal']
deficit_surplus = 'surplus'
if market_balance < 0:
    deficit_surplus = 'deficit'
market_balance = "{:,}".format(abs(market_balance))
prev_mkt_bal = df_market_bal[(df_market_bal.Month == current_month_num) & (
    df_market_bal['Year'] == last_year)]['YTDMarketBal'].values[0]
prev_def_sur = 'surplus'
if prev_mkt_bal < 0:
    prev_def_sur = 'deficit'
prev_mkt_bal = "{:,}".format((abs(prev_mkt_bal)))
prev_year_whole = df_market_bal[(
    df_market_bal['Year'] == last_year)]['MarketBalance'].sum()
prev_year_def_sur = 'surplus'
if prev_year_whole < 0:
    prev_year_def_sur = 'deficit'
prev_year_whole = "{:,}".format(abs(prev_year_whole))

The below code calls the relevant template for Zinc market balance commentary based on the month.

In [14]:
if current_month_num == '01':
    fin_text = sent_jan(metal_name, 'January', calendar.month_name[int(
        current_month_num)], production, consumption, deficit_surplus, market_balance, metal, howmanymonths, prev_mkt_bal, prev_def_sur, prev_year_whole)
elif current_month_num == '12':
    fin_text = sent_dec(metal_name, calendar.month_name[int(
        current_month_num)], production, consumption, deficit_surplus, market_balance, prev_year_whole, prev_year_def_sur)
else:
    fin_text = (sent_non_jan_non_dec(metal_name, 'January', calendar.month_name[int(
        current_month_num)], production, consumption, deficit_surplus, market_balance, metal, howmanymonths, prev_mkt_bal, prev_def_sur, prev_year_whole, prev_year_def_sur))
fin_text_zinc = fin_text.replace(".0", "")
fin_text_zinc

'Refined Lead output for the January to December period, was 13,864,807 tonnes compared to apparent consumption of 13,665,709 tonnes, producing an implied market surplus of 199,098 tonnes in the market for the current year.\nThe 199,098 tonnes surplus compares with 94,883 tonnes deficit for the previous year.'

In the below cells we consolidate the market balance commentary for the six base metals and generate a .txt file.

In [15]:
lst_text = fin_text_aluminium.replace("/n", "").split('.')+['\n']+fin_text_copper.replace("\n", "").split('.')+['\n']+fin_text_lead.replace("\n", "").split(
    '.')+['\n']+fin_text_nickel.replace("\n", "").split('.')+['\n']+fin_text_tin.replace("\n", "").split('.')+['\n']+fin_text_zinc.replace("\n", "").split('.')+['\n']
lst_new_text = []
for i in lst_text:
    i = i.replace("*", ".")
    lst_new_text.append(i)

In [16]:
%%capture cap --no-stderr
for i in lst_new_text:
    print(i)

In [17]:
from pathlib import Path
with open(Path.home()/'WBMSMarketBalanceAutomatedCommentary.txt', 'w') as f:
    f.write(cap.stdout)

In [18]:
rd.close_session()